In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle


In [ ]:
!cp kaggle.json ~/.kaggle

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d chandrashekarnatesh/poultry-diseases

Dataset URL: https://www.kaggle.com/datasets/chandrashekarnatesh/poultry-diseases
License(s): MIT
... resuming from 6165626880 bytes (7339603769 bytes left) ...
100% 12.6G/12.6G [05:56<00:00, 25.2MB/s]
100% 12.6G/12.6G [05:56<00:00, 20.6MB/s]


In [ ]:
!unzip poultry-diseases


Streaming output truncated to the last 5000 lines.
  inflating: data/data/val/Salmonella/salmo.1854.jpg_aug27.JPG  
  inflating: data/data/val/Salmonella/salmo.1854.jpg_aug3.JPG  
  inflating: data/data/val/Salmonella/salmo.1855.jpg_aug15.JPG  
  inflating: data/data/val/Salmonella/salmo.1855.jpg_aug27.JPG  
  inflating: data/data/val/Salmonella/salmo.1855.jpg_aug29.JPG  
  inflating: data/data/val/Salmonella/salmo.1855.jpg_aug45.JPG  
  inflating: data/data/val/Salmonella/salmo.1856.jpg_aug11.JPG  
  inflating: data/data/val/Salmonella/salmo.1856.jpg_aug12.JPG  
  inflating: data/data/val/Salmonella/salmo.1856.jpg_aug14.JPG  
  inflating: data/data/val/Salmonella/salmo.1856.jpg_aug18.JPG  
  inflating: data/data/val/Salmonella/salmo.1856.jpg_aug20.JPG  
  inflating: data/data/val/Salmonella/salmo.1856.jpg_aug46.JPG  
  inflating: data/data/val/Salmonella/salmo.1857.jpg_aug11.JPG  
  inflating: data/data/val/Salmonella/salmo.1857.jpg_aug14.JPG  
  inflating: data/data/val/Salmonella/sa

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2
import pandas as pd

In [ ]:
IMAGE_SIZE = (224, 224,3)


In [ ]:
train_data= "/content/data/data/train"
val_data = "/content/data/data/val"
test_data= "/content/data/data/test"

In [ ]:
labels = os.listdir(train_data)

In [ ]:
labels

['Healthy', 'New Castle Disease', 'Coccidiosis', 'Salmonella']

In [ ]:
def read_data(folder):
    data, label, paths = [], [], []
    for l in labels:
        path = f"{folder}/{l}/"
        folder_data = os.listdir(path)[:500]
        for image_path in folder_data:
            img = cv2.imread(path + image_path)
            data.append(img)
            label.append(l)
            paths.append(os.path.join(folder, l, image_path))

    return data, label, paths

In [ ]:
all_data, all_labels, all_paths = read_data(train_data)

In [ ]:
train_df = pd.DataFrame({
    'image':all_data,
    'path': all_paths,
    'label': all_labels
})

In [ ]:
train_df

,image,path,label
0,"[[[172, 172, 172], [177, 177, 177], [175, 175,...",/content/data/data/train/Healthy/healthy.1392....,Healthy
1,"[[[191, 184, 175], [185, 178, 169], [192, 185,...",/content/data/data/train/Healthy/healthy.1034....,Healthy
2,"[[[123, 123, 123], [124, 124, 124], [123, 123,...",/content/data/data/train/Healthy/healthy.617.j...,Healthy
3,"[[[180, 179, 169], [183, 182, 172], [188, 187,...",/content/data/data/train/Healthy/healthy.1849....,Healthy
4,"[[[106, 106, 106], [92, 92, 92], [95, 95, 95],...",/content/data/data/train/Healthy/healthy.1960....,Healthy
...,...,...,...
1995,"[[[56, 59, 50], [70, 71, 62], [67, 65, 57], [6...",/content/data/data/train/Salmonella/pcrsalmo.9...,Salmonella
1996,"[[[169, 165, 170], [163, 159, 164], [182, 179,...",/content/data/data/train/Salmonella/pcrsalmo.1...,Salmonella
1997,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",/content/data/data/train/Salmonella/pcrsalmo.1...,Salmonella
1998,"[[[200, 212, 216], [203, 214, 218], [206, 215,...",/content/data/data/train/Salmonella/salmo.1637...,Salmonella


In [ ]:
all_data, all_labels, all_paths = read_data(test_data)

In [ ]:
test_df = pd.DataFrame({
    'image':all_data,
    'path': all_paths,
    'label': all_labels
})

In [ ]:
all_data, all_labels, all_paths = read_data(val_data)

In [ ]:
val_df = pd.DataFrame({
    'image':all_data,
    'path': all_paths,
    'label': all_labels
})

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
gen = ImageDataGenerator()

In [ ]:
train_gen=gen.flow_from_dataframe(train_df, x_col='path', y_col='label', target_size=(224,224),seed=123,
                                   class_mode='categorical', color_mode='rgb', shuffle=True, batch_size=32)

Found 2000 validated image filenames belonging to 4 classes.


In [ ]:
test_gen=gen.flow_from_dataframe(test_df, x_col='path', y_col='label', target_size=(224,224),seed=123,
                                   class_mode='categorical', color_mode='rgb', shuffle=True, batch_size=32)

Found 2000 validated image filenames belonging to 4 classes.


In [ ]:
val_gen=gen.flow_from_dataframe(val_df, x_col='path', y_col='label', target_size=(224, 224),seed=123,
                                   class_mode='categorical', color_mode='rgb', shuffle=True, batch_size=32)

Found 2000 validated image filenames belonging to 4 classes.


In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(vgg.output)
prediction = Dense(4, activation='softmax')(x)


In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
from tensorflow.keras.optimizers.legacy import Adam

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']

)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
r = model.fit(
  train_gen,
  validation_data=val_gen,
  epochs=5,
)

Epoch 1/5
63/63 [==============================] - 2295s 37s/step - loss: 9.0686 - accuracy: 0.4915 - val_loss: 9.0997 - val_accuracy: 0.5400
Epoch 2/5
63/63 [==============================] - 2356s 38s/step - loss: 2.0549 - accuracy: 0.8175 - val_loss: 7.6841 - val_accuracy: 0.5915
Epoch 3/5
55/63 [=========================>....] - ETA: 2:23 - loss: 0.5620 - accuracy: 0.9106

In [ ]:
from tensorflow.keras.applications import VGG19

In [ ]:
vgg1 = VGG19(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)

In [ ]:
for layer in vgg1.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(vgg1.output)
prediction = Dense(4, activation='softmax')(x)


In [ ]:
model1 = Model(inputs=vgg1.input, outputs=prediction)

In [ ]:
model1.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']

)

In [ ]:
r = model1.fit(
  train_gen,
  validation_data=val_gen,
  epochs=5,
)

In [ ]:
from tensorflow.keras.applications import ResNet50

In [ ]:
res = ResNet50(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)

In [ ]:
for layer in res.layers:
  layer.trainable = False

x = Flatten()(res.output)
prediction = Dense(4, activation='softmax')(x)


In [ ]:
model2 = Model(inputs=res.input, outputs=prediction)

In [ ]:
model2.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']

)

In [ ]:
r = model1.fit(
  train_gen,
  validation_data=val_gen,
  epochs=5,
)

In [ ]:
model.predict(test_gen)